# Rekko challenge 2019

```
                           /$$$$$$$  /$$$$$$$$ /$$   /$$ /$$   /$$  /$$$$$$ 
                          | $$__  $$| $$_____/| $$  /$$/| $$  /$$/ /$$__  $$
                          | $$  \ $$| $$      | $$ /$$/ | $$ /$$/ | $$  \ $$
                          | $$$$$$$/| $$$$$   | $$$$$/  | $$$$$/  | $$  | $$
                          | $$__  $$| $$__/   | $$  $$  | $$  $$  | $$  | $$
                          | $$  \ $$| $$      | $$\  $$ | $$\  $$ | $$  | $$
                          | $$  | $$| $$$$$$$$| $$ \  $$| $$ \  $$|  $$$$$$/
                          |__/  |__/|________/|__/  \__/|__/  \__/ \______/ 
                                                                            
```

Добро пожаловать на соревнование по машинному обучению от онлайн-кинотеатра [Okko](http://okko.tv) Rekko Challenge 2019.

В этом ноутбуке мы покажем вам пример простого но полного решения, от загрузки данных до формирования ответа. Для работы нам понадобятся библиотеки `pandas`, `numpy`, `scipy`, `implicit`, `pprint`, `tqdm`. Установить их в вашем рабочем окружении можно следующей командой.
```
pip install pandas numpy scipy implicit pprint tqdm
```

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
from collections import defaultdict
import pickle

from pprint import pprint

### Загрузка данных

Замените `DATA_PATH` на путь к данным, которые вы скачали со страницы соревнования.

In [ ]:
DATA_PATH = '/mnt/data/molchanov/datasets/okko/rekko_challenge_rekko_challenge_2019'

---

`catalogue.json` содержит анонимизированную метаинформацию о доступных в сервисе фильмах и сериалах.

In [ ]:
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}

In [ ]:
pprint(catalogue[13])

In [ ]:
catalogue2 = defaultdict(list)
for key, val in catalogue.items():
    if 'purchase' in val['availability']:
        catalogue2['purchase'].append(1)
    else:
        catalogue2['purchase'].append(0)
        
    if 'rent' in val['availability']:
        catalogue2['rent'].append(1)
    else:
        catalogue2['rent'].append(0)
        
    if 'subscription' in val['availability']:
        catalogue2['subscription'].append(1)
    else:
        catalogue2['subscription'].append(0)
        
    catalogue2['element_uid'].append(key)
    catalogue2['feature_1'].append(val['feature_1'])
    catalogue2['feature_2'].append(val['feature_2'])
    catalogue2['feature_3'].append(val['feature_3'])
    catalogue2['feature_4'].append(val['feature_4'])
    catalogue2['feature_5'].append(val['feature_5'])
    catalogue2['duration'].append(val['duration'])
    catalogue2['type'].append(val['type'])

df_catalogue = pd.DataFrame({
    'element_uid': catalogue2['element_uid'],
    
    'feature_1': catalogue2['feature_1'],
    'feature_2': catalogue2['feature_2'],
    'feature_3': catalogue2['feature_3'],
    'feature_4': catalogue2['feature_4'],
    'feature_5': catalogue2['feature_5'],
    
    'type': catalogue2['type'],
    'duration': catalogue2['duration'],
    
    'purchase': catalogue2['purchase'],
    'rent': catalogue2['rent'],
    'subscription': catalogue2['subscription'],
})

df_catalogue['feature_1'] = df_catalogue['feature_1'].astype(np.float64)
df_catalogue['feature_2'] = df_catalogue['feature_2'].astype(np.float64)
df_catalogue['feature_3'] = df_catalogue['feature_3'].astype('category').cat.codes
df_catalogue['feature_4'] = df_catalogue['feature_4'].astype(np.float64)
df_catalogue['feature_5'] = df_catalogue['feature_5'].astype(np.float64)
df_catalogue['element_uid'] = df_catalogue['element_uid'].astype(np.uint16)
df_catalogue['duration'] = df_catalogue['duration'].astype(np.uint64)
df_catalogue['type'] = df_catalogue['type'].astype('category').cat.codes
df_catalogue.head(10)

 - `attributes` — мешок атрибутов
 - `availability` — доступность (может содержать значения `purchase`, `rent` и `subscription`)
 - `duration` — длительность в минутах, округлённая до десятков (продолжительность серии для сериалов и многосерийных фильмов)
 - `feature_1..5` — пять анонимизированных вещественных и порядковых признаков
 - `type` — принимает значения `movie`, `multipart_movie` или `series`

---

In [ ]:
df_catalogue['element_uid'].values.max()

`test_users.json` содержит список пользователей, для которых необходимо построить предсказание

In [ ]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

---

`transactions.csv` — список всех транзакций за определённый период времени

In [ ]:
%%time
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)


In [ ]:
transactions['device_manufacturer'].values.max()

 - `element_uid` — идентификатор элемента
 - `user_uid` — идентификатор пользователя
 - `consumption_mode` — тип потребления (`P` — покупка, `R` — аренда, `S` — просмотр по подписке)
 - `ts` — время совершения транзакции или начала просмотра в случае просмотра по подписке
 - `watched_time` — число просмотренных по транзакции секунд
 - `device_type` — анонимизированный тип устройства, с которого была совершена транзакция или начат просмотр
 - `device_manufacturer` — анонимизированный производитель устройства, с которого была совершена транзакция или начат просмотр

In [ ]:
dataset = None

transactions_ex = pd.merge(transactions, df_catalogue, on='element_uid', how='left')

columns = ['element_uid', 
           'consumption_mode', 
           'device_type', 
           'device_manufacturer', 
           'watched_time',
           'feature_1',
           'feature_2',
           'feature_3',
           'feature_4',
           'feature_5',
           'type',
           'duration',
           'purchase',
           'rent',
           'subscription'
          ]
main_col = 'user_uid'

transactions_ex['consumption_mode'] = transactions_ex['consumption_mode'].astype('category').cat.codes

for group_col in tqdm.tqdm(columns):
    tmp = transactions_ex.sort_values([main_col,'ts'],ascending=False).\
                                    groupby(['user_uid'])[group_col].\
                                    apply(list).\
                                    reset_index()
    if dataset is None:
        dataset = tmp
    else:
        dataset = pd.merge(dataset, tmp, on=main_col)
dataset.head(5)

In [ ]:
dataset.to_csv(os.path.join(DATA_PATH, 'transactions_grouped.csv'))

In [ ]:
dataset = pd.read_csv(os.path.join(DATA_PATH, 'transactions_grouped.csv'))
dataset.head(5)

In [ ]:
import ast
columns = ['element_uid', 'consumption_mode',
       'device_type', 'device_manufacturer', 'watched_time', 'feature_1',
       'feature_2', 'feature_3', 'feature_4', 'feature_5', 'type', 'duration',
       'purchase', 'rent', 'subscription']

pickle_dataset = []
for index, row in dataset.iterrows():
    feature_arrays = dict()
    for col in columns:
        feature_arrays[col] =  ast.literal_eval(row[col])
        
    itm = dict()
    itm['user_uid'] = row['user_uid']
    itm['len'] = len(feature_arrays['element_uid'])
    itm['feature_arrays'] = feature_arrays
    
    pickle_dataset.append(itm)
    
pickle_dataset[0]

In [ ]:
for val in pickle_dataset:
    if 'type' in val['feature_arrays']:
        val['feature_arrays']['video_type'] = val['feature_arrays']['type']
        val['feature_arrays'].pop('type', None)
pickle_dataset[0]

In [ ]:
with open(os.path.join(DATA_PATH, 'transactions_grouped.pkl'), 'wb') as f:
    pickle.dump(pickle_dataset, f)

In [ ]:
with open(os.path.join(DATA_PATH, 'transactions_grouped.pkl'), 'rb') as f:
    pickle_dataset = pickle.load(f)
pickle_dataset[0]

In [ ]:
# lens histogram
import matplotlib.pyplot as plt
l = []
for v in pickle_dataset:
    if v['len'] > 200:
        continue
    l.append(v['len'])
    
plt.hist(l, bins=300)
plt.show()

In [ ]:
l = [i for i in range(100)]
np.random.randint(0, 2, (100,))

---

`ratings.csv` содержит информацию о поставленных пользователями оценках

In [ ]:
%%time
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

In [ ]:
ratings.head(3)

 - `rating` — поставленный пользователем рейтинг (от `0` до `10`)

---

`bookmarks.csv` содержит информацию об элементах, добавленных пользователями в список «Избранное»

In [ ]:
%%time
bookmarks = pd.read_csv(
    os.path.join(DATA_PATH, 'bookmarks.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64
    }
)

In [ ]:
bookmarks.head(3)

### Решение

Для начала построим список элементов, которые тестовые пользователи уже купили или посмотрели по подписке: они не смогут купить их второй раз, а просмотр по подписке второй раз маловероятен, поэтому мы захотим отфильтровать такие элементы из финального ответа.

Точно так же можно поступить и с рейтингами и добавлениями в избранное, если это будет казаться правильным.

In [ ]:
from collections import defaultdict

filtered_elements = defaultdict(set)

for user_uid, element_uid in tqdm.tqdm(transactions.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)

---

Для примера мы воспользуемся методом K ближайших соседей, реализованным в библиотеке `implicit`. В качестве данных используем только информацию о рейтингах.

Необходимо построить разреженную матрицу, где строкам будут соответствовать элементы, столбцам — пользователи, а на пересечении пользователя и элемента будет находиться количественная оценка степени их взаимодействия, если таковое имело место.

Не забудем добавить `1` к рейтингу, чтобы избежать деления на ноль во время вычисления `tf-idf`.

In [ ]:
ratings['user_uid'] = ratings['user_uid'].astype('category')
ratings['element_uid'] = ratings['element_uid'].astype('category')

ratings_matrix = sp.coo_matrix(
    (ratings['rating'].astype(np.float32) + 1,
        (
            ratings['element_uid'].cat.codes.copy(),
            ratings['user_uid'].cat.codes.copy()
        )
    )
)

ratings_matrix = ratings_matrix.tocsr()

In [ ]:
sparsity = ratings_matrix.nnz / (ratings_matrix.shape[0] * ratings_matrix.shape[1])
print('Sparsity: %.6f' % sparsity)

Обучить модель крайне просто.

In [ ]:
from implicit.nearest_neighbours import TFIDFRecommender

model = TFIDFRecommender()
model.fit(ratings_matrix)

---

In [ ]:
ratings_matrix_T = ratings_matrix.T.tocsr()

Отображения из оригинальной категории во внутреннюю пригодится нам в дальнейшем.

In [ ]:
user_uid_to_cat = dict(zip(
    ratings['user_uid'].cat.categories,
    range(len(ratings['user_uid'].cat.categories))
))

In [ ]:
element_uid_to_cat = dict(zip(
    ratings['element_uid'].cat.categories,
    range(len(ratings['element_uid'].cat.categories))
))

In [ ]:
filtered_elements_cat = {k: [element_uid_to_cat.get(x, None) for x in v] for k, v in filtered_elements.items()}

---

В метод `model.recommend` мы передаём идентификатор пользователя, который получаем обратным преобразованием из категории, транспонированную матрицу взаимодействий, число необходимых рекомендаций и список элементов, которые мы договорились фильтровать из ответа.

Возвращает метод список пар (`element_cat`, `score`), отсортированный по вторым элементам. Из него необходимо достать все первые элементы пар и из категории преобразовать их к `element_uid`.

**Важно:** Не все тестовые пользователи есть в `ratings.csv` и не все из них есть в `transactions.csv`. Используя только один источник данных мы не можем построить полное предсказание. Такой ответ с неполным числом пользователей бдет принят системой, но при вычислении средней метрики метрика для отсутствующих пользователей будет принята равной нулю.

In [ ]:
result = {}

for user_uid in tqdm.tqdm(test_users):
    # transform user_uid to model's internal user category
    try:
        user_cat = user_uid_to_cat[user_uid]
    except LookupError:
        continue
    
    # perform inference
    recs = model.recommend(
        user_cat,
        ratings_matrix_T,
        N=20,
        filter_already_liked_items=True,
        filter_items=filtered_elements_cat.get(user_uid, set())
    )
    
    # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    result[user_uid] = [int(ratings['element_uid'].cat.categories[i]) for i, _ in recs]

In [ ]:
len(result)

Используя только информацию о рейтингах мы смогли построить предсказание для `13251` из `50000` тестовых пользователей. Ровно в таком виде ответы и стоит сохранить для отправки.

In [ ]:
with open('answer.json', 'w') as f:
    json.dump(result, f)